In [1]:
import json
import requests


def request_get(url, param):
    fails = 0
    while True:
        try:
            if fails >= 20:
                break

            ret = requests.get(url=url, params=param, timeout=10)

            if ret.status_code == 200:
                text = json.loads(ret.text)
            else:
                continue
        except:
            fails += 1
            print('网络连接出现问题, 正在尝试再次请求: ', fails)
        else:
            break
    return text


def request_post(url, param):
    fails = 0
    while True:
        try:
            if fails >= 20:
                break

            headers = {'content-type': 'application/json'}
            ret = requests.post(url, json=param, headers=headers, timeout=10)

            if ret.status_code == 200:
                text = json.loads(ret.text)
            else:
                continue
        except:
            fails += 1
            print('网络连接出现问题, 正在尝试再次请求: ', fails)
        else:
            break
    return text


In [2]:
pre_url = 'https://dmfw.mca.gov.cn/9095/stname/listPub?stName='
end_url = '&type=place&page=1&size=1'

data = request_post(pre_url + '虎丘区' + end_url, {})
data['records'][0]['area']


def get_code(name):
    data = request_post(pre_url + name + end_url, {})
    if data and 'records' in data and len(data['records']) == 1 and 'area' in data['records'][0]:
        return data['records'][0]['area']
    else:
        return '-'

print(data)


{'total': 22, 'records': [{'id': '7c59929a-44b3-46be-989f-2db4a471029e', 'place_code': '32050510421400000000', 'standard_name': '虎丘区', 'roman_alphabet_spelling': 'Hǔqiū Qū', 'ethnic_minorities_writing': '', 'place_type': '县级行政区', 'place_type_code': '21400', 'province_name': '江苏省', 'city_name': '苏州市', 'area_name': '虎丘区', 'area': '320505', 'city': '3205', 'province': '32', 'gdm': {'type': 'multipoint', 'coordinates': [[120.4293428, 31.3314431]]}, 'pdm': None}]}


In [5]:
import os
os.getcwd()


'e:\\paper\\code\\script\\excel数据处理'

In [11]:
import json
from time import sleep
from openpyxl import load_workbook

# Load in the workbook
src_dir = r'E:\paper\data\原始数据\历年行政区划数据'
wb = load_workbook(src_dir + '\\' + '行政区划变更列表.xlsx')
# src_dir = os.getcwd()
# wb = load_workbook(src_dir + '/' + 'temp.xlsx')


# Create empty list
data_list = []
id = 1

# Loop through each sheet in the workbook
for sheet in wb.worksheets:
    # Iterate through each row in the sheet  
    for row in sheet.iter_rows():
        row_data = {}

        # Iterate through each cell in the row
        for cell in row:
            # Get the value of the first row
            row_heading = sheet.cell(1, cell.column).value

            # Update the row_data dictionary with row heading as key and the value of the cell
            if cell.value:
              row_data[row_heading] = str(cell.value)

        # Add administrative code
        if 'originCode' not in row_data:
          row_data['originCode'] = get_code(row_data['originName'])
          row_data['newCode'] = get_code(row_data['newName'])
        # Append the row_data into the data_list
        if cell.row != 1:
          row_data['id'] = id
          id += 1
          data_list.append(row_data)

# # Serialize the data_list into JSON string
# j = json.dumps(data_list, ensure_ascii='utf-8')
# # Print the JSON string
# print(data_list)

In [12]:
data_list

[{'originName': '徽州地区',
  'originCode': '342700',
  'changeTime': '11月27日国务院批复同意 国函〔1987〕185号[1]',
  'changeContent': '撤销徽州地区，设立黄山市（地级）',
  'newName': '黄山市',
  'newCode': '341000',
  'changeType': '地级市增设',
  'year': '1987',
  'id': 1},
 {'originName': '屯溪市',
  'originCode': '342701',
  'changeTime': '11月27日国务院批复同意 国函〔1987〕185号[1]',
  'changeContent': '撤销屯溪市，设立黄山市屯溪区，其行政区域包括原屯溪市、歙县和休宁县部分乡',
  'newName': '屯溪区',
  'newCode': '341002',
  'changeType': '地级市增设',
  'year': '1987',
  'id': 2},
 {'originName': '歙县',
  'originCode': '342723',
  'changeTime': '11月27日国务院批复同意 国函〔1987〕185号[1]',
  'changeContent': '设立黄山市徽州区，其行政区域包括歙县部分乡镇',
  'newName': '徽州区',
  'newCode': '341004',
  'changeType': '地级市增设',
  'year': '1987',
  'id': 3},
 {'originName': '歙县',
  'originCode': '342723',
  'changeTime': '11月27日国务院批复同意 国函〔1987〕185号[1]',
  'changeContent': '划归黄山市管辖',
  'newName': '歙县',
  'newCode': '341021',
  'changeType': '地级市增设',
  'year': '1987',
  'id': 4},
 {'originName': '休宁县',
  'originCode': '34272

In [10]:
with open(src_dir + '/' + '行政区划变更.json', 'w+', encoding='utf-8') as f:
  json.dump(data_list, f, ensure_ascii=False)